# Test
A notebook for interactive validation of the data inserted into Cassandra. Will not work if the following have not been completed in this order:

1. Run a Cassandra Docker Container (full instructions in the README)

```
docker run --name cassandra-container -p 127.0.0.1:9042:9042 -d cassandra
```
2. Run **create_tables.py**
3. Run **etl.py**

Close the cluster connection at the bottom of this notebook when done.

## Setup

In [4]:
import pandas as pd
from cassandra.cluster import Cluster
from cql_queries import *

In [9]:
# Create a connection to the cassandra docker container, set the keyspace
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)
    print('Did you run create_tables.py and etl.py prior to this?')

This is a little something that will help visualize the queries as dataframes.

In [7]:
# Define function that will instantiate a dataframe row factory
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

def query_to_df(query, session):
    """
    Returns
    -------
    A pandas DataFrame with the query results.
    
    Parameters
    ----------
    query: A CQL query to select data from a given table.
    
    session: A connection to the Cassandra cluster.
    """
    # Specify to use the pandas_factory
    session.row_factory = pandas_factory

    # Default is 50000, but our tables are pretty small so ignore
    session.default_fetch_size = None

    # Execute query 
    rows = session.execute(query)
    df = rows._current_rows
    
    return df

## `song_info` Table

**Query:** Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [5]:
print(select_query_1)


    SELECT
      artist,
      song,
      length
    FROM
      song_info
    WHERE
      session_id = 338 AND
      item_in_session = 4



In [10]:
# Confirm data was inserted
query_to_df(select_query_1, session)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## `users_songs` Table

**Query:** Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [6]:
print(select_query_2)


    SELECT
      artist,
      song,
      first_name,
      last_name
    FROM
      users_songs
    WHERE
      user_id = 10 AND
      session_id = 182



In [11]:
# Confirm data was inserted
query_to_df(select_query_2, session)

,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## `user_name` Table

**Query:** Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [8]:
print(select_query_3)


    SELECT
      first_name,
      last_name
    FROM
      user_name
    WHERE
      song = 'All Hands Against His Own'



In [12]:
# Confirm data was inserted
query_to_df(select_query_3, session)

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [10]:
# You can also drop the tables if you want
for query in drop_table_queries:
    session.execute(query)

In [11]:
session.shutdown()
cluster.shutdown()